In [33]:
#importing library
import pandas as pd
import requests
from bs4 import BeautifulSoup
import numpy as np
import re
import nltk
import string
import os
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

#nltk.download('stopwords')

#nltk.download('punkt')

In [31]:
#importing input file
df=pd.read_excel('Input.xlsx')
#loop throgh each row in the df
for index, row in df.iterrows():
    url = row['URL']
    url_id = row['URL_ID']

    # make a request to url
    header = {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36"}
    try:
        response = requests.get(url,headers=header)
    except:
        print("can't get response of {}".format(url_id))

    #create a beautifulsoup object
    try:
        soup = BeautifulSoup(response.content, 'html.parser')
    except:
        print("can't get page of {}".format(url_id))
    #find title
    try:
        title = soup.find('h1').get_text()
    except:
        print("can't get title of {}".format(url_id))
        continue
    #find text
    article = ""
    try:
        for p in soup.find_all('p'):
            article += p.get_text()
    except:
        print("can't get text of {}".format(url_id))

    #write title and text to the file
    file_name = 'Text/'+ str(url_id) + '.txt'
    with open(file_name, 'w',encoding = 'UTF8', errors ='replace') as file:
        file.write(title + '\n' + article)
        
    
        

can't get title of blackassign0036
can't get title of blackassign0049


In [38]:
#all the folders are stored in one place only
text_dir = "Text"
stopwords_dir = "StopWords"
sentment_dir = "MasterDictionary"

# load all stop words from the stopwords directory and store in the set variable
stop_words = set()
for files in os.listdir(stopwords_dir):
    with open(os.path.join(stopwords_dir,files),'r',encoding='ISO-8859-1',  errors ='replace') as f:
        stop_words.update(set(f.read().splitlines()))

# load all text files  from the  directory and store in a list(docs)
docs = []
for text_file in os.listdir(text_dir):
    with open(os.path.join(text_dir,text_file),'r',  errors ='replace') as f:
        text = f.read()
        #tokenize the given text file
        words = word_tokenize(text)
        # remove the stop words from the tokens
        filtered_text = [word for word in words if word.lower() not in stop_words]
        # add each filtered tokens of each file into a list
        docs.append(filtered_text)



# store positive, Negative words from the directory
pos=set()
neg=set()

for files in os.listdir(sentment_dir):
    if files =='positive-words.txt':
        with open(os.path.join(sentment_dir,files),'r',encoding='ISO-8859-1',  errors ='replace') as f:
            pos.update(f.read().splitlines())
    else:
        with open(os.path.join(sentment_dir,files),'r',encoding='ISO-8859-1',  errors ='replace') as f:
            neg.update(f.read().splitlines())

# now collect the positive  and negative words from each file
# calculate the scores from the positive and negative words 
positive_words = []
Negative_words =[]
positive_score = []
negative_score = []
polarity_score = []
subjectivity_score = []

#iterate through the list of docs
for i in range(len(docs)):
    positive_words.append([word for word in docs[i] if word.lower() in pos])
    Negative_words.append([word for word in docs[i] if word.lower() in neg])
    positive_score.append(len(positive_words[i]))
    negative_score.append(len(Negative_words[i]))
    polarity_score.append((positive_score[i] - negative_score[i]) / ((positive_score[i] + negative_score[i]) + 0.000001))
    subjectivity_score.append((positive_score[i] + negative_score[i]) / ((len(docs[i])) + 0.000001))


In [44]:

# Average Sentence Length = the number of words / the number of sentences
# Percentage of Complex words = the number of complex words / the number of words 
# Fog Index = 0.4 * (Average Sentence Length + Percentage of Complex words)

avg_sentence_length = []
Percentage_of_Complex_words  =  []
Fog_Index = []
complex_word_count =  []
avg_syllable_word_count =[]

def measure(file):
    with open(os.path.join(text_dir, file),'r', errors ='replace') as f:
        text = f.read()
        # remove punctuations 
        text = re.sub(r'[^\w\s.]','',text)
        # split the given text file into sentences
        sentences = text.split('.')
        # total number of sentences in a file
        num_sentences = len(sentences)
        # total words in the file
        words = [word  for word in text.split() if word.lower() not in stopwords ]
        num_words = len(words)
 
        # complex words having syllable count is greater than 2
        # Complex words are words in the text that contain more than two syllables.
        complex_words = []
        for word in words:
            vowels = 'aeiou'
            syllable_count_word = sum( 1 for letter in word if letter.lower() in vowels)
            if syllable_count_word > 2:
                complex_words.append(word)

# Syllable Count Per Word
# We count the number of Syllables in each word of the text by counting the vowels present in each word.
#  We also handle some exceptions like words ending with "es","ed" by not counting them as a syllable.
    syllable_count = 0
    syllable_words =[]
    for word in words:
        if word.endswith('es'):
            word = word[:-2]
        elif word.endswith('ed'):
            word = word[:-2]
        vowels = 'aeiou'
        syllable_count_word = sum( 1 for letter in word if letter.lower() in vowels)
        if syllable_count_word >= 1:
            syllable_words.append(word)
            syllable_count += syllable_count_word


    avg_sentence_len = num_words / num_sentences
    avg_syllable_word_count = syllable_count / len(syllable_words)
    Percent_Complex_words  =  len(complex_words) / num_words
    Fog_Index = 0.4 * (avg_sentence_len + Percent_Complex_words)

    return avg_sentence_len, Percent_Complex_words, Fog_Index, len(complex_words),avg_syllable_word_count

# iterate through each file or doc
for file in os.listdir(text_dir):
    x,y,z,a,b = measure(file)
    avg_sentence_length.append(x)
    Percentage_of_Complex_words.append(y)
    Fog_Index.append(z)
    complex_word_count.append(a)
    avg_syllable_word_count.append(b)

In [46]:
# Word Count and Average Word Length Sum of the total number of characters in each word/Total number of words
# We count the total cleaned words present in the text by 
# removing the stop words (using stopwords class of nltk package).
# removing any punctuations like ? ! , . from the word before counting.

def cleaned_words(file):
    with open(os.path.join(text_dir,file), 'r', errors ='replace') as f:
        text = f.read()
        text = re.sub(r'[^\w\s]', '' , text)
        words = [word  for word in text.split() if word.lower() not in stopwords]
        length = sum(len(word) for word in words)
        average_word_length = length / len(words)
    return len(words),average_word_length

word_count = []
average_word_length = []
for file in os.listdir(text_dir):
    x, y = cleaned_words(file)
    word_count.append(x)
    average_word_length.append(y)


# To calculate Personal Pronouns mentioned in the text, we use regex to find 
# the counts of the words - “I,” “we,” “my,” “ours,” and “us”. Special care is taken
#  so that the country name US is not included in the list.
def count_personal_pronouns(file):
    with open(os.path.join(text_dir,file), 'r', errors ='replace') as f:
        text = f.read()
        personal_pronouns = ["I", "we", "my", "ours", "us"]
        count = 0
        for pronoun in personal_pronouns:
            count += len(re.findall(r"\b" + pronoun + r"\b", text)) # \b is used to match word boundaries
    return count

pp_count = []
for file in os.listdir(text_dir):
    x = count_personal_pronouns(file)
    pp_count.append(x)

In [57]:
output_df = pd.read_excel('Output Data Structure.xlsx')

# URL_ID 36 ,49 does not exists i,e. page does not exist, throughs 404 error
# so we are going to drop these rows from the table
output_df.drop([36,49], axis = 0, inplace=True)

# These are the required parameters 
variables = [positive_score,
            negative_score,
            polarity_score,
            subjectivity_score,
            avg_sentence_length,
            Percentage_of_Complex_words,
            Fog_Index,
            avg_sentence_length,
            complex_word_count,
            word_count,
            avg_syllable_word_count,
            pp_count,
            average_word_length]

# write the values to the dataframe
for i, var in enumerate(variables):
    output_df.isetitem(i+2, var)

#now save the dataframe to the disk
output_df.to_csv('Output_Data.csv')
     

In [69]:
check = pd.read_csv("Output_Data.csv")

check

,Unnamed: 0,URL_ID,URL,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCE,COMPLEX WORD COUNT,WORD COUNT,SYLLABLE PER WORD,PERSONAL PRONOUNS,AVG WORD LENGTH
0,0,blackassign0001,https://insights.blackcoffer.com/rising-it-cit...,10,4,0.428571,0.037634,10.290323,0.473354,4.305471,10.290323,151,316,2.694079,2,6.933544
1,1,blackassign0002,https://insights.blackcoffer.com/rising-it-cit...,61,32,0.311828,0.084469,11.571429,0.565844,4.854909,11.571429,550,972,2.811702,4,7.369342
2,2,blackassign0003,https://insights.blackcoffer.com/internet-dema...,42,27,0.217391,0.074919,13.049180,0.615578,5.465903,13.049180,490,796,3.049032,14,8.097990
3,3,blackassign0004,https://insights.blackcoffer.com/rise-of-cyber...,41,75,-0.293103,0.128319,14.035714,0.591603,5.850927,14.035714,465,785,2.981603,5,7.959236
4,4,blackassign0005,https://insights.blackcoffer.com/ott-platform-...,23,11,0.352941,0.057143,12.000000,0.509259,5.003704,12.000000,275,540,2.789675,7,7.433333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,95,blackassign0096,https://insights.blackcoffer.com/what-is-the-r...,31,57,-0.295455,0.106280,13.928571,0.492308,5.768352,13.928571,384,780,2.689564,3,7.160256
94,96,blackassign0097,https://insights.blackcoffer.com/impact-of-cov...,25,37,-0.193548,0.090511,14.840909,0.468606,6.123806,14.840909,306,652,2.559748,6,6.792945
95,97,blackassign0098,https://insights.blackcoffer.com/contribution-...,5,3,0.250000,0.027972,25.444444,0.532751,10.390878,25.444444,122,229,2.760369,1,7.262009
96,98,blackassign0099,https://insights.blackcoffer.com/how-covid-19-...,16,5,0.523809,0.043478,12.911765,0.421412,5.333271,12.911765,185,436,2.541568,3,6.724771


In [70]:
df=pd.read_excel('Input.xlsx')

In [71]:
df.drop('URL_ID',axis=1,inplace=True)

In [75]:
url_id=1
for i in range(0,len(df)):
    j=df.iloc[i].values
    headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36'}#giving user access
    page=requests.get(j[0],headers=headers)#loading text in url
    soup=BeautifulSoup(page.content,'html.parser')#parsing url text
    content=soup.findAll(attrs={'class':'td-post-content'})#extracting only text part
    content=content[0].text.replace('\xa0',"  ").replace('\n',"  ")#replace end line symbol with space 
    title=soup.findAll(attrs={'class':'entry-title'})#extracting title of website
    title=title[16].text.replace('\n',"  ").replace('/',"")
    text=title+ '.' +content#merging title and content text
    text=np.array(text)#converting to array form
    text.reshape(1,-1)#changing shape to 1d 
    df1=pd.Series(text)#creating series data frame
    b=str(url_id)+"."+'txt'#name of the text file
    with open(b, 'a', errors ='replace') as f:#creating text file 
        df1.to_csv(f, lineterminator=',', index=False, header=False)
  # files.download(b)#downloading text file
    url_id+=1

C:\Users\nehar\AppData\Local\Temp\ipykernel_15384\2733516913.py:17: FutureWarning: the 'line_terminator'' keyword is deprecated, use 'lineterminator' instead.
  df1.to_csv(f, line_terminator=',', index=False, header=False)


IndexError: list index out of range